In [1]:
def loadLibFolder (folder):
    import os, sys
    if folder not in sys.path:
        sys.path.insert(1, os.path.join(sys.path[0], folder))

# Experimenting with POS dependency parser
To be able to predict a category out of a sentence/text it is assumed that the POS tags and the dependency tree could have an inpact on the result. Here we investigate that relation

In [2]:
from urllib import request, parse
import json
url = 'http://localhost:1337/sentence/'

## Sample text to try out the parser

In [3]:
def parseSentence(sentence):
    try:
        sentence = request.quote(sentence)
        f =  request.urlopen(url + sentence)
        res = json.loads(f.read().decode('latin1'))
        return res
    except:
        return {'sentenceData': []}
def onlyNounsAndVerbs(data):
    return {
        'sentenceData': [word for word in data['sentenceData'] if 'NN' in word['tag'].split('|') or 'VB' in word['tag'].split('|')]
    }
def untilLevel(level, data):
    return {
        'sentenceData': [word for word in data['sentenceData'] if (int)(word['parent']) <= level]
    }
def toWordArray(data):
    return [word['base_word'] for word in data['sentenceData']]

In [4]:
res = parseSentence('Han ler mot henne och hela hans ansikte säger att han älskar henne med hela sitt hjärta')

In [5]:
# Example filtering
print ("Raw data:")
print (res)
print ("All words:")
print ([word['word'] for word in res['sentenceData']])
print ("Level three data:")
print ([word['word']+ '::' + word['tag'].split('|')[0] for word in res['sentenceData'] if (int)(word['parent']) <= 3])
print ("Only nouns and verbs:")
print ([word['word'] for word in res['sentenceData'] if 'NN' in word['tag'].split('|') or 'VB' in word['tag'].split('|')])

print(" ".join(toWordArray(untilLevel(3, onlyNounsAndVerbs(parseSentence(res))))))

Raw data:
{'sentenceData': [{'identifier': '1', 'word': 'han', 'base_word': 'han', 'tag': 'PN|UTR|SIN|DEF|SUB', 'parent': '2'}, {'identifier': '2', 'word': 'ler', 'base_word': 'le', 'tag': 'VB|PRS|AKT', 'parent': '0'}, {'identifier': '3', 'word': 'mot', 'base_word': 'mot', 'tag': 'PP', 'parent': '2'}, {'identifier': '4', 'word': 'henne', 'base_word': 'hon', 'tag': 'PN|UTR|SIN|DEF|OBJ', 'parent': '3'}, {'identifier': '5', 'word': 'och', 'base_word': 'och', 'tag': 'KN', 'parent': '8'}, {'identifier': '6', 'word': 'hela', 'base_word': 'hel', 'tag': 'JJ|POS|UTR/NEU|SIN|DEF|NOM', 'parent': '8'}, {'identifier': '7', 'word': 'hans', 'base_word': 'hans', 'tag': 'PS|UTR/NEU|SIN/PLU|DEF', 'parent': '8'}, {'identifier': '8', 'word': 'ansikte', 'base_word': 'ansikte', 'tag': 'NN|NEU|SIN|IND|NOM', 'parent': '9'}, {'identifier': '9', 'word': 'säger', 'base_word': 'säga', 'tag': 'VB|PRS|AKT', 'parent': '3'}, {'identifier': '10', 'word': 'att', 'base_word': 'att', 'tag': 'SN', 'parent': '9'}, {'identi

## Classification experiment

In [6]:
loadLibFolder('../gensim')

import os
import gensim
import gensim_documents
import dotenv
import numpy as np
dotenv.load()

C:\Users\desktop-godesity\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [62]:
limit_per_category = 1000
use_cache = True

In [ ]:
categories = []
x_data = []
y_data = []
model = gensim.models.Doc2Vec.load(dotenv.get('DOC2VEC_MODEL'))

if use_cache and os.path.isfile('data/tmp_dependency_data'):
    with open('data/tmp_dependency_data', 'r', encoding='utf-8', errors='ignore') as tmp_cache_file:
        for category in tmp_cache_file:
            category = category[:-1]
            if category == "\n": continue
            if category not in categories:
                print ("TT", category)
                categories.append(category)
            sentVecs = []
            while True:
                sentence = tmp_cache_file.readline()[:-1]
                if sentence == "":
                    break
                artvec = model.infer_vector(doc_words=sentence.split())
                sentVecs.append(gensim.matutils.unitvec(artvec))
            y_data.append(categories.index(category))
            x_data.append(sentVecs)
else:
    data = gensim_documents.MMDBDocumentLists(dotenv.get('ARTICLE_PATH', '.') + '/csv_by_category/', useHeading=True, limit=limit_per_category)
    with open('data/tmp_dependency_data', 'w', encoding='utf-8', errors='ignore') as tmp_cache_file:
        for i, doc in enumerate(data):
            if not doc.category in categories:
                categories.append(doc.category)
            tmp_cache_file.write(doc.category + "\n")

            sentences = doc.content.split(".")
            sentVecs = []
            for j in range(20):
                if j >= len(sentences): 
                    sentVecs.append(np.zeros(300))
                    continue
                sentence = " ".join(toWordArray(untilLevel(3, onlyNounsAndVerbs(parseSentence(sentences[j])))))
                if sentence == "":
                    sentVecs.append(np.zeros(300))
                    continue
                artvec = model.infer_vector(doc_words=sentence.split())
                sentVecs.append(gensim.matutils.unitvec(artvec))
                tmp_cache_file.write(sentence + "\n")
            tmp_cache_file.write("\n")
            x_data.append(sentVecs)
            y_data.append(categories.index(doc.category))

            if i % limit_per_category == 0:
                print ("New epoch started, nr.", i+1, " of ", len(categories) * limit_per_category, " epochs")


New epoch started, nr. 1  of  7  epochs
New epoch started, nr. 2  of  7  epochs
New epoch started, nr. 3  of  7  epochs
New epoch started, nr. 4  of  7  epochs
New epoch started, nr. 5  of  7  epochs
New epoch started, nr. 6  of  7  epochs


#### Encode one hot vectors for the classes

In [52]:
y_data_one_hot = np.zeros((len(y_data), len(categories)))
y_data_one_hot[np.arange(len(y_data)), np.array(y_data)] = 1


### LSTM classification with keras LSTM cells

In [53]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
import numpy as np

In [54]:
data_dim = len(x_data[0][0])
timesteps = len(x_data[0])
num_classes = len(categories)

In [55]:
split = 0.4
# Generate dummy training data
x_train = x_data[:(int)(len(x_data) * split)]
y_train = y_data_one_hot[:(int)(len(x_data) * split)]

# Generate dummy validation data
x_val = x_data[(int)(len(x_data) * split):]
y_val = y_data_one_hot[(int)(len(x_data) * split):]

In [56]:
print(len(x_val))
print(len(x_val[0]))
print(len(x_val[0][0]))

print(len(x_train))
print(len(x_train[0]))
print(len(x_train[0][0]))

print(y_train)

26
20
300
16
20
300
[[ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.]]


To train a Sequential LSTM model that can classify a stacked sequence of words we need to define the input as follows:
 * batch_size - number of datapoints in the dataset
 * timesteps - the number of words per sequence
 * data_dim - the number of features per word instance

In [57]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(num_classes, activation='softmax'))
#model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

Train on 16 samples, validate on 26 samples
Epoch 1/5
16/16 [==============================] - 8s - loss: 1.9467 - acc: 0.1250 - val_loss: 1.9460 - val_acc: 0.2692
Epoch 2/5
16/16 [==============================] - 8s - loss: 1.9347 - acc: 0.4375 - val_loss: 1.9467 - val_acc: 0.2692
Epoch 3/5
16/16 [==============================] - 8s - loss: 1.9227 - acc: 0.5000 - val_loss: 1.9483 - val_acc: 0.2692
Epoch 4/5
16/16 [==============================] - 8s - loss: 1.9058 - acc: 0.3750 - val_loss: 1.9521 - val_acc: 0.3077
Epoch 5/5
16/16 [==============================] - 8s - loss: 1.8808 - acc: 0.3125 - val_loss: 1.9589 - val_acc: 0.3077


In [58]:
model.evaluate(x_val, y_val)

26/26 [==============================] - 1s


[1.9588522911071777, 0.30769231915473938]